# Инсталляция библиотек

In [6]:
# Инсталляция библиотеки для отображения финансовых диаграмм
!pip install --upgrade mplfinance

Requirement already up-to-date: mplfinance in /usr/local/lib/python3.6/dist-packages (0.12.7a5)


In [7]:
# Инсталляция билиотеки для технического анализа
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

(Reading database ... 146400 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



# Загружаем библиотеки

In [8]:
from urllib.parse import urlencode
from urllib.request import urlopen
from datetime import datetime

# Библиотеки для отображения
from matplotlib import pyplot as plt
# import seaborn as sns
# import statsmodels.tsa.api as smt
# import statsmodels.api as sm
from mplfinance.original_flavor import candlestick_ohlc

# Numpy & Pandas
import numpy as np
from pandas import pandas as pd, DataFrame as df

# System Libs
import os
import io
# from IPython.display import display
# import datetime
from __future__ import print_function
#from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, HTML, HBox, VBox, Button, Dropdown, DatePicker, Layout, GridspecLayout
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, HTML, HBox, VBox, Button, Dropdown, DatePicker, Layout, GridspecLayout

# Библиотека для технического анализа
import talib

# Вспомогательные функции

## Функции для отображения диаграмм

In [ ]:
def basic_diagram(fig_size=(22,10)):
  fig = plt.figure(figsize=fig_size)
  ax = fig.add_subplot(111)
  ax.grid()
  return ax


def add_lines(stock, ax, col_name, start=0, frame_len='All', color='red'):
  data = stock.copy()
  stop = (frame_len, data.shape[0])[int(frame_len=='All')]
  data = data.iloc[start: start + stop].reset_index(drop=True)

  ax.plot(data.index, data[col_name], color=color, label=col_name)
  return ax



def add_discrets(stock, ax, col_name, start=0, frame_len='All', color=['red', 'green']):
  data = stock.copy()
  stop = (frame_len, data.shape[0])[int(frame_len=='All')]
  data = data.iloc[start: start + stop].reset_index(drop=True)

  y_min, y_max = ax.get_ylim()
  delta = (y_max - y_min) * 0.1 / 2
  for idx, item in data[col_name].items():
    if item != 0:
      sign = (0, 1)[int(item > 0)]
      clr = color[sign]
      ax.vlines(idx, y_min + delta, y_max - delta, color=clr)
  return ax



def add_candles(stock, ax, start=0, frame_len='All'):
  data = stock.copy()
  stop = (frame_len, data.shape[0])[int(frame_len=='All')]
  data = data.iloc[start : start + stop].reset_index(drop=True)

  quotes = [tuple([
                idx,
                data['open'].iloc[idx], 
                data['high'].iloc[idx], 
                data['low'].iloc[idx], 
                data['close'].iloc[idx]
                ]) for idx in data.index]
  
  candlestick_ohlc(ax, quotes, colordown='r', colorup='g', alpha=0.5)
  return ax


def add_ticks(stock, ax, col_name, start=0, frame_len='All', shift=5, color=['red', 'green']):
  data = stock.copy()
  stop = (frame_len, data.shape[0])[int(frame_len=='All')]
  data = data.iloc[start: start + stop].reset_index(drop=True)

  min = data[['high', 'low']].min().min()
  max = data[['high', 'low']].max().max()
  shift = (max - min) * (shift / 100)

  for idx, item in data[col_name].items():
    if item !=0:
      y = (data['high'].iloc[idx] + shift, data['low'].iloc[idx] - shift)[int(item > 0)]
      sign = ('v', '^')[int(item > 0)]
      clr = color[int(item > 0)]
      ax.plot(idx, y, sign, color=clr)

  return ax


def show_diagram(legend=True):
  if legend:
    plt.legend()
  #plt.grid()
  plt.show()

# Определение стратегии торговли

In [ ]:
# Сеть будет предсказывать какое действие лучше совершить покупку/продажу/никакое
# Стратегия строится на торговле на long и short сделках
#
# Для long сделок выделяется бюджет money, на который каждый раз при появлении сигнала на покупку совершается покупка n акций
# Все купленные акции продаются при первом же появлении сигнала к продаже. 
#
# Для short сделок определяется кредитный базис - это сумма, в пределах которой продаются акции брокера, если нет собственных 
# Купленные на short сделке акции продаются при следующем сигнале на продажу, в случае, если цена продажи стала ниже, чем была при покупке.
# В момент продажи акций по кредитной short сделке обязательно уменьшается кредитная поддержка, чтобы в случае, 
# если поведение рынка будет непредсказуемым не совершать сделки, которые будут ничем не обеспечены (бесконечный долг перед брокером).
# 
# При каждом сигнале на продажу продаются все собственные акции, а в случае выгодной для сделки по кредитным акциям 
# выполняется проверка аккумулятора денег (money) и если сумма средств от сделки позволяет, то восстанавливается уровень кредитной поддержки до базового или до стартового (макс уровень)
#

def trading_strategy(tr_data, money=50000, shares=0, basic_loan_security=50000, action_col='order', bear_margin_wait=1.1, bull_margin_wait=1.3, fix_margin=21, stop_loss_steps=21, full_history=False, verbose=False, only_actions=True):
  actions = {0: 'none', 1: 'buy', -1:'sell'}
  loan_security = basic_loan_security
  loan_shares = 0
  loan_shares_price = 0
  action_done = 0
  stop_loss = 0
  margin_steps = 0
  shares_price = 0

  money_history = df(columns=['action', 'money', 'own_shares', 'cur_price', 'loan_security', 'loan_shares', 'loan_shares_price'])

  items = tr_data[[action_col, 'close']].values

  for item in items:
    action, price = item[0], item[1]
    
    if action == 1 and money > price:
      shares += money // price # Покупаем на все имеющиеся деньги акции (покупаем целое количество)
      money = money % price # Считаем сколько осталось денег после покупки акций
      action_done = 1
      shares_price = price

    # Если должны продавать
    elif action == -1:
      if shares > 0:
        if shares_price * bull_margin_wait <= price or margin_steps > fix_margin:
          # Продаем все акции и увеличиваем капитал на заработанные деньги
          # Количество акцией сбрасываем в 0
          action_done = -1
          sell_value = shares * price 
          shares = 0
          money += sell_value       

      if loan_shares > 0:
        if loan_shares_price >= price * bear_margin_wait or stop_loss > stop_loss_steps:
          loan_shares = 0
          sell_value = loan_shares * (loan_shares_price - price)
          money += sell_value
          loan_security += money * 0.5
          money *= 0.5
          stop_loss = 0
          action_done = -1

      elif loan_shares == 0 and loan_security >= price:
        loan_shares = loan_security // price 
        sell_value = loan_shares * price
        loan_shares_price = price
        money += sell_value
        loan_security -= sell_value
        action_done = -1

    else:
      action = 0
      action_done = 0
      if not only_actions:
        if stop_loss > stop_loss_steps and loan_shares > 0:
          loan_shares = 0
          sell_value = loan_shares * (loan_shares_price - price)
          money += sell_value
          loan_security += money * 0.5
          money *= 0.5
          stop_loss = 0
        if margin_steps > fix_margin and shares > 0:
          action_done = -1
          sell_value = shares * price 
          shares = 0
          money += sell_value

    stop_loss += 1
    margin_steps += 1

    money_history = money_history.append(
        {'action': actions[action], 
         'action_done': action_done,
         'money': round(money, 2), 
         'own_shares': shares, 
         'shares_price': shares_price,
         'current_price': price, 
         'basic_loan_security': basic_loan_security, 
         'loan_security': loan_security, 
         'loan_shares': loan_shares, 
         'loan_shares_price': loan_shares_price,
         'total_assets': round(money + loan_security + shares * price + (loan_shares * loan_shares_price - loan_shares * price),2),
         'stop_loss': stop_loss,
         'margin_steps': margin_steps
        }, 
        ignore_index=True)

  if verbose:
    print(money_history)

  return (money_history.iloc[-1], money_history)[int(full_history)]

In [ ]:
def trading_strategy_1(tr_data, money=50000, basic_loan_security=50000, action_col='order', bear_margin_wait=1.1, bull_margin_wait=1.3, fix_margin=21, stop_loss_steps=21, full_history=False, verbose=False, only_actions=True):
  actions = {0: 'none', 1: 'buy', -1:'sell'}
  loan_security = basic_loan_security
  loan_shares = 0
  loan_shares_price = 0
  action_done = 0
  stop_loss = 0
  margin_steps = 0
  shares_price = 0
  shares=0

  money_history = df(columns=['action', 'money', 'own_shares', 'cur_price', 'loan_security', 'loan_shares', 'loan_shares_price'])

  items = tr_data[[action_col, 'close']].values

  for item in items:
    action, price = item[0], item[1]
    
    if action == 1 and money >= price:
      shares += money // price # Покупаем на все имеющиеся деньги акции (покупаем целое количество)
      money = money % price # Считаем сколько осталось денег после покупки акций
      action_done = 1
      shares_price = price

    # Если должны продавать
    elif action == -1:
      if shares > 0:
        action_done = -1
        sell_value = shares * price 
        shares = 0
        money += sell_value       

    else:
      action = 0
      action_done = 0

    stop_loss += 1
    margin_steps += 1

    money_history = money_history.append(
        {'action': actions[action], 
         'action_done': action_done,
         'money': round(money, 2), 
         'own_shares': shares, 
         'shares_price': shares_price,
         'current_price': price, 
         'basic_loan_security': basic_loan_security, 
         'loan_security': loan_security, 
         'loan_shares': loan_shares, 
         'loan_shares_price': loan_shares_price,
         'total_assets': round(money + loan_security + shares * price + (loan_shares * loan_shares_price - loan_shares * price),2),
         'stop_loss': stop_loss,
         'margin_steps': margin_steps
        }, 
        ignore_index=True)

  if verbose:
    print(money_history)

  return (money_history.iloc[-1], money_history)[int(full_history)]

# Загружаем котировки

In [ ]:
#markets = [{ value: 200, title: 'ММВБ-Top' }, { value: 1, title: 'ММВБ Акции' }, { value: 29, title: 'ММВБ ПИФы' }, { value: 2, title: 'ММВБ облигации' }, { value: 12, title: 'ММВБ Внесписочные облигации' }, { value: 8, title: 'АДР' }, { value: 6, title: 'Мировые Индексы' }, { value: 24, title: 'Товары' }, { value: 5, title: 'Forex' }, { value: 14, title: 'Фьючерсы ФОРТС' }, { value: 3, title: 'РТС' }, { value: 38, title: 'RTS Standard' }, { value: 20, title: 'RTS Board' }, { value: 10, title: 'РТС-GAZ' }, { value: 25, title: 'Акции США(BATS)' }, { value: 7, title: 'Фьючерсы США' }, { value: 27, title: 'Отрасли экономики США' }, { value: 26, title: 'Гособлигации США' }, { value: 28, title: 'ETF' }, { value: 30, title: 'Индексы мировой экономики' }, { value: 17, title: 'ФОРТС Архив' }, { value: 31, title: 'Сырье Архив' }, { value: 16, title: 'ММВБ Архив' }, { value: 18, title: 'РТС Архив' }, { value: 9, title: 'СПФБ Архив' }, { value: 39, title: 'АДР Архив' }]

In [ ]:
periods={'tick': 1, 'min': 2, '5min': 3, '10min': 4, '15min': 5, '30min': 6, 'hour': 7, 'daily': 8, 'week': 9, 'month': 10}

tickers={'ABRD':82460,'AESL':181867,'AFKS':19715,'AFLT':29,'AGRO':399716,'AKRN':17564,
         'ALBK':82616,'ALNU':81882,'ALRS':81820,'AMEZ':20702,'APTK':13855,'AQUA':35238,
				 'ARMD':19676,'ARSA':19915,'ASSB':16452,'AVAN':82843,'AVAZ':39,'AVAZP':40,
				 'BANE':81757,'BANEP':81758,'BGDE':175840,'BISV':35242,'BISVP':35243,'BLNG':21078,
				 'BRZL':81901,'BSPB':20066,'CBOM':420694,'CHEP':20999,'CHGZ':81933,'CHKZ':21000,
				 'CHMF':16136,'CHMK':21001,'CHZN':19960,'CLSB':16712,'CLSBP':16713,'CNTL':21002,
				 'CNTLP':81575,'DASB':16825,'DGBZ':17919,'DIOD':35363,'DIXY':18564,'DVEC':19724,
				 'DZRD':74744,'DZRDP':74745,'ELTZ':81934,'ENRU':16440,'EPLN':451471,'ERCO':81935,
				 'FEES':20509,'FESH':20708,'FORTP':82164,'GAZA':81997,'GAZAP':81998,'GAZC':81398,
				 'GAZP':16842,'GAZS':81399,'GAZT':82115,'GCHE':20125,'GMKN':795,'GRAZ':16610,
				 'GRNT':449114,'GTLC':152876,'GTPR':175842,'GTSS':436120,'HALS':17698,'HIMC':81939,
				 'HIMCP':81940,'HYDR':20266,'IDJT':388276,'IDVP':409486,'IGST':81885,'IGST03':81886,
				 'IGSTP':81887,'IRAO':20516,'IRGZ':9,'IRKT':15547,'ISKJ':17137,'JNOS':15722,
				 'JNOSP':15723,'KAZT':81941,'KAZTP':81942,'KBSB':19916,'KBTK':35285,'KCHE':20030,
				 'KCHEP':20498,'KGKC':83261,'KGKCP':152350,'KLSB':16329,'KMAZ':15544,'KMEZ':22525,
				 'KMTZ':81903,'KOGK':20710,'KRKN':81891,'KRKNP':81892,'KRKO':81905,'KRKOP':81906,
				 'KROT':510,'KROTP':511,'KRSB':20912,'KRSBP':20913,'KRSG':15518,'KSGR':75094,
				 'KTSB':16284,'KTSBP':16285,'KUBE':522,'KUNF':81943,'KUZB':83165,'KZMS':17359,
				 'KZOS':81856,'KZOSP':81857,'LIFE':74584,'LKOH':8,'LNTA':385792,'LNZL':21004,
				 'LNZLP':22094,'LPSB':16276,'LSNG':31,'LSNGP':542,'LSRG':19736,'LVHK':152517,
				 'MAGE':74562,'MAGEP':74563,'MAGN':16782,'MERF':20947,'MFGS':30,'MFGSP':51,
				 'MFON':152516,'MGNT':17086,'MGNZ':20892,'MGTS':12984,'MGTSP':12983,'MGVM':81829,
				 'MISB':16330,'MISBP':16331,'MNFD':80390,'MOBB':82890,'MOEX':152798,'MORI':81944,
				 'MOTZ':21116,'MRKC':20235,'MRKK':20412,'MRKP':20107,'MRKS':20346,'MRKU':20402,
				 'MRKV':20286,'MRKY':20681,'MRKZ':20309,'MRSB':16359,'MSNG':6,'MSRS':16917,
				 'MSST':152676,'MSTT':74549,'MTLR':21018,'MTLRP':80745,'MTSS':15523,'MUGS':81945,
				 'MUGSP':81946,'MVID':19737,'NAUK':81992,'NFAZ':81287,'NKHP':450432,'NKNC':20100,
				 'NKNCP':20101,'NKSH':81947,'NLMK':17046,'NMTP':19629,'NNSB':16615,'NNSBP':16616,
				 'NPOF':81858,'NSVZ':81929,'NVTK':17370,'ODVA':20737,'OFCB':80728,'OGKB':18684,
				 'OMSH':22891,'OMZZP':15844,'OPIN':20711,'OSMP':21006,'OTCP':407627,'PAZA':81896,
				 'PHOR':81114,'PHST':19717,'PIKK':18654,'PLSM':81241,'PLZL':17123,'PMSB':16908,
				 'PMSBP':16909,'POLY':175924,'PRFN':83121,'PRIM':17850,'PRIN':22806,'PRMB':80818,
				 'PRTK':35247,'PSBR':152320,'QIWI':181610,'RASP':17713,'RBCM':74779,'RDRB':181755,
				 'RGSS':181934,'RKKE':20321,'RLMN':152677,'RLMNP':388313,'RNAV':66644,'RODNP':66693,
				 'ROLO':181316,'ROSB':16866,'ROSN':17273,'ROST':20637,'RSTI':20971,'RSTIP':20972,
				 'RTGZ':152397,'RTKM':7,'RTKMP':15,'RTSB':16783,'RTSBP':16784,'RUAL':414279,
				 'RUALR':74718,'RUGR':66893,'RUSI':81786,'RUSP':20712,'RZSB':16455,'SAGO':445,
				 'SAGOP':70,'SARE':11,'SAREP':24,'SBER':3,'SBERP':23,'SELG':81360,'SELGP':82610,
				 'SELL':21166,'SIBG':436091,'SIBN':2,'SKYC':83122,'SNGS':4,'SNGSP':13,'STSB':20087,
				 'STSBP':20088,'SVAV':16080,'SYNG':19651,'SZPR':22401,'TAER':80593,'TANL':81914,
				 'TANLP':81915,'TASB':16265,'TASBP':16266,'TATN':825,'TATNP':826,'TGKA':18382,
				 'TGKB':17597,'TGKBP':18189,'TGKD':18310,'TGKDP':18391,'TGKN':18176,'TGKO':81899,
				 'TNSE':420644,'TORS':16797,'TORSP':16798,'TRCN':74561,'TRMK':18441,'TRNFP':1012,
				 'TTLK':18371,'TUCH':74746,'TUZA':20716,'UCSS':175781,'UKUZ':20717,'UNAC':22843,
				 'UNKL':82493,'UPRO':18584,'URFD':75124,'URKA':19623,'URKZ':82611,'USBN':81953,
				 'UTAR':15522,'UTII':81040,'UTSY':419504,'UWGN':414560,'VDSB':16352,'VGSB':16456,
				 'VGSBP':16457,'VJGZ':81954,'VJGZP':81955,'VLHZ':17257,'VRAO':20958,'VRAOP':20959,
				 'VRSB':16546,'VRSBP':16547,'VSMO':15965,'VSYD':83251,'VSYDP':83252,'VTBR':19043,
				 'VTGK':19632,'VTRS':82886,'VZRZ':17068,'VZRZP':17067,'WTCM':19095,'WTCMP':19096,
				 'YAKG':81917,'YKEN':81766,'YKENP':81769,'YNDX':388383,'YRSB':16342,'YRSBP':16343,
				 'ZHIV':181674,'ZILL':81918,'ZMZN':556,'ZMZNP':603,'ZVEZ':82001}

In [ ]:
def click_OK(b):
  global source_data

  market = 0
  start_date = start_d.value
  end_date = end_d.value  
  
  start_date_str = f'{start_date.year}-{start_date.month-1}-{start_date.day}'
  end_date_str = f'{end_date.year}-{end_date.month-1}-{end_date.day}'
  
  FINAM_URL = "http://export.finam.ru/"# сервер, на который стучимся
  params = urlencode([
            ('market', market), #на каком рынке торгуется бумага
            ('em', ticker.value), #вытягиваем цифровой символ, который соответствует бумаге.
            ('code', ticker.label), #тикер нашей акции
            ('apply',0), #не нашёл что это значит. 
            ('df', start_date.day), #Начальная дата, номер дня (1-31)
            ('mf', start_date.month - 1), #Начальная дата, номер месяца (0-11)
            ('yf', start_date.year), #Начальная дата, год
            ('from', start_date_str), #Начальная дата полностью
            ('dt', end_date.day), #Конечная дата, номер дня	
            ('mt', end_date.month - 1), #Конечная дата, номер месяца
            ('yt', end_date.year), #Конечная дата, год
            ('to', end_date_str), #Конечная дата
            ('p', period.value), #Таймфрейм
            ('f', f'{ticker.label}_{start_date_str}_{end_date_str}'), #Имя сформированного файла
            ('e', ".csv"), #Расширение сформированного файла
            ('cn', ticker.value), #ещё раз тикер акции	
            ('dtf', 1), #В каком формате брать даты. Выбор из 5 возможных. См. страницу https://www.finam.ru/profile/moex-akcii/sberbank/export/
            ('tmf', 1), #В каком формате брать время. Выбор из 4 возможных.
            ('MSOR', 0), #Время свечи (0 - open; 1 - close)	
            ('mstime', "on"), #Московское время	
            ('mstimever', 1), #Коррекция часового пояса	
            ('sep', 1), #Разделитель полей	(1 - запятая, 2 - точка, 3 - точка с запятой, 4 - табуляция, 5 - пробел)
            ('sep2', 1), #Разделитель разрядов
            ('datf', 1), #Формат записи в файл. Выбор из 6 возможных.
            ('at', 1) #Нужны ли заголовки столбцов
            ]) 

  url = f'{FINAM_URL}{ticker.label}_{start_date_str}_{end_date_str}.csv?{params}'

  print(f'Стучимся на Финам за {ticker.label}')
  source_data = pd.read_csv(url, sep=',')
  source_data.columns = ['ticker', 'per', 'date', 'time', 'open', 'high', 'low', 'close', 'vol']
  path = '/content/drive/MyDrive/Colab Notebooks/Трейдинг/data'
  file_name = f'{path}/{ticker.label}_{start_date_str}_{end_date_str}.csv'
  source_data.to_csv(file_name)
  print(source_data.head(5))

  print(f"Готово! Данные сохранены здесь: {file_name}")

In [ ]:
date_e = datetime.today()
date_s = datetime(date_e.year - 1, date_e.month, date_e.day)

start_d = DatePicker(value=date_s,  description='Start: ')
end_d = DatePicker(value=date_e, description='End: ')

period = Dropdown(options=periods, index=6, description='Periods: ')
ticker = Dropdown(options=tickers, index=list(tickers.keys()).index('SBER'), description='Tickers: ')

OK = Button(description='Download', layout=Layout(height='auto', width='auto'))

layout = GridspecLayout(n_rows=4, n_columns=3, width='auto', height='auto', )
layout[0, 0] = HBox([start_d, end_d], width='auto', height='auto')
layout[1, 0] = HBox([ticker, period], layout=Layout(height='auto', width='auto'))
layout[3, 0] = OK

# Загрузка


In [ ]:
display(layout)

OK.on_click(click_OK)

GridspecLayout(children=(HBox(children=(DatePicker(value=datetime.datetime(2020, 1, 22, 0, 0), description='St…

Стучимся на Финам за SBER
   ticker  per      date    time    open    high     low   close      vol
0       3   60  20200122  100000  268.63  270.80  268.39  269.93  9192840
1       3   60  20200122  110000  269.93  270.69  268.05  268.27  6427020
2       3   60  20200122  120000  268.19  268.59  266.84  268.02  5958530
3       3   60  20200122  130000  268.05  268.82  267.41  267.68  2677420
4       3   60  20200122  140000  267.68  267.69  265.50  265.51  4499580
Готово! Данные сохранены здесь: /content/drive/MyDrive/Colab Notebooks/Трейдинг/data/SBER_2020-0-22_2021-0-22.csv
Стучимся на Финам за SBER
   ticker  per      date    time    open    high     low   close      vol
0       3   60  20200122  100000  268.63  270.80  268.39  269.93  9192840
1       3   60  20200122  110000  269.93  270.69  268.05  268.27  6427020
2       3   60  20200122  120000  268.19  268.59  266.84  268.02  5958530
3       3   60  20200122  130000  268.05  268.82  267.41  267.68  2677420
4       3   60  202

In [ ]:
class trading():
  def __init__(self, broker_percent=0.004):
    self.shares = 0
    self.money = 100000
    self.broker_percent = broker_percent

  def exec_order(self, order, price):
    money_in = self.money
    shares_in = self.shares
    if order == 1 and self.money >= price:
      self.shares += self.money // (price + price * self.broker_percent) # Покупаем на все имеющиеся деньги акции (покупаем целое количество)
      self.money = self.money % price # Считаем сколько осталось денег после покупки акций

    # Если должны продавать
    elif order == -1:
      if self.shares > 0:
        sell_value = self.shares * price 
        self.shares = 0
        self.money += sell_value - sell_value * self.broker_percent


    return [money_in, shares_in, self.money, self.shares]
    


In [ ]:
def create_data(data, forward_steps=10, backward_steps=10, use_backward=False, use_forward=True, use_patterns=False, patterns_presense=0.1, use_acceleration=False, order_col='order'):
  trader = trading()

  # Индикаторы
  if use_patterns:
    open, close, high, low = data['open'], data['close'], data['high'], data['low']
    ext_cdl = []
    cdl = [item for item in talib.__TA_FUNCTION_NAMES__ if 'CDL' in item]
    for item in cdl:
      col = getattr(talib, item)(open, high, low, close)
      if col[col != 0].count() >= data.shape[0] * patterns_presense:
        data[item.lower()] = col / 100
      else:
        ext_cdl.append(item)
    cdl = [item for item in cdl if item not in ext_cdl]

  if use_forward:
    steps = forward_steps # Кол-во шагов вперед
    d_cols = []
    for step in range(1, steps):
      data[f'd{-1*(step-steps)}'] = data['close'].shift(step-steps) # Будущие тики
      data[f'd{-1*(step-steps)}'] = data['close'] - data[f'd{-1*(step-steps)}']
      d_cols.append(f'd{-1*(step-steps)}')
    data['pos'] = data[d_cols].apply(lambda x: (-1, 1)[int(abs(x.max()) > abs(x.min()))], axis=1)
    data['pos_d1'] = data['pos'].shift(1)
    data['order'] = data[['pos', 'pos_d1']].apply(lambda x: (x['pos'], 0)[int(x['pos'] == x['pos_d1'])],axis=1)

  if use_forward:
    steps = backward_steps
    d_cols_2 = []
    for step in range(1, steps):
      data[f'd-{step}'] = data['close'].shift(step) # Прошлые тики
      data[f'd-{step}'] = data['close'] - data[f'd-{step}']
      d_cols_2.append(f'd-{step}')
    data['pos_2'] = data[d_cols_2].apply(lambda x: (-1, 1)[int(abs(x.max()) > abs(x.min()))], axis=1)
    data['pos_d1_2'] = data['pos_2'].shift(1)
    data['order_2'] = data[['pos_2', 'pos_d1_2']].apply(lambda x: (x['pos_2'], 0)[int(x['pos_2'] == x['pos_d1_2'])],axis=1)

  if use_acceleration:
    dd_cols, dd_cols_2 = [], []
    for step in range(1, steps):
      data[f'dd-{step}'] = data['d-1'].shift(step) # Прошлые тики
      data[f'dd-{step}'] = data['d-1'] - data[f'dd-{step}']
      dd_cols.append(f'dd-{step}')

    data['pos_3'] = data[dd_cols].apply(lambda x: (-1, 1)[int(abs(x.max()) > abs(x.min()))], axis=1)
    data['pos_dd1'] = data['pos_3'].shift(1)
    data['order_3'] = data[['pos_3', 'pos_dd1']].apply(lambda x: (x['pos_3'], 0)[int(x['pos_3'] == x['pos_dd1'])],axis=1)

  data['money_in', 'shares_in', 'money_out', 'shares_out'] = data.apply(lambda x: trader.exec_order(x[order_col], x['close']), axis=1)
  #data['shares_in'] = data.apply(lambda x: trader.exec_order(x[order_col], x['close'])[1], axis=1)
  #data['money_out'] = data.apply(lambda x: trader.exec_order(x[order_col], x['close'])[2], axis=1)
  #data['shares_out'] = data.apply(lambda x: trader.exec_order(x[order_col], x['close'])[3], axis=1)
  return data

try:
  del data
  data = source_data.copy()
except:
  data = source_data.copy()

data = create_data(source_data.copy())
data

,ticker,per,date,time,open,high,low,close,vol,d9,d8,d7,d6,d5,d4,d3,d2,d1,pos,pos_d1,order,d-1,d-2,d-3,d-4,d-5,d-6,d-7,d-8,d-9,pos_2,pos_d1_2,order_2,"(money_in, shares_in, money_out, shares_out)"
0,3,60,20200122,100000,268.63,270.80,268.39,269.93,9192840,4.63,3.39,2.58,3.18,3.70,4.42,2.25,1.91,1.66,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,-1.0,"[100000, 0, 125.89999999999748, 368.0]"
1,3,60,20200122,110000,269.93,270.69,268.05,268.27,6427020,2.27,2.97,1.73,0.92,1.52,2.04,2.76,0.59,0.25,1,1.0,0.0,-1.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1.0,0.0,"[125.89999999999748, 368.0, 125.89999999999748..."
2,3,60,20200122,120000,268.19,268.59,266.84,268.02,5958530,3.06,2.02,2.72,1.48,0.67,1.27,1.79,2.51,0.34,1,1.0,0.0,-0.25,-1.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1.0,0.0,"[125.89999999999748, 368.0, 125.89999999999748..."
3,3,60,20200122,130000,268.05,268.82,267.41,267.68,2677420,2.78,2.72,1.68,2.38,1.14,0.33,0.93,1.45,2.17,1,1.0,0.0,-0.34,-0.59,-2.25,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1.0,0.0,"[125.89999999999748, 368.0, 125.89999999999748..."
4,3,60,20200122,140000,267.68,267.69,265.50,265.51,4499580,-0.68,0.61,0.55,-0.49,0.21,-1.03,-1.84,-1.24,-0.72,-1,1.0,-1.0,-2.17,-2.51,-2.76,-4.42,NaN,NaN,NaN,NaN,NaN,-1,-1.0,0.0,"[125.89999999999748, 368.0, 97442.74927999999, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,3,60,20210121,210000,274.59,275.01,274.57,274.94,368070,NaN,NaN,NaN,NaN,NaN,6.43,2.88,0.54,-0.05,1,1.0,0.0,0.38,-0.22,-0.81,1.40,0.53,-0.10,0.62,-0.16,0.00,1,-1.0,1.0,"[117.96927999994637, 4.0, 117.96927999994637, ..."
2997,3,60,20210121,220000,274.95,275.20,274.60,274.99,460200,NaN,NaN,NaN,NaN,NaN,NaN,6.48,2.93,0.59,1,1.0,0.0,0.05,0.43,-0.17,-0.76,1.45,0.58,-0.05,0.67,-0.11,1,1.0,0.0,"[117.96927999994637, 4.0, 117.96927999994637, ..."
2998,3,60,20210121,230000,275.00,275.09,274.36,274.40,642200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.89,2.34,1,1.0,0.0,-0.59,-0.54,-0.16,-0.76,-1.35,0.86,-0.01,-0.64,0.08,-1,1.0,-1.0,"[117.96927999994637, 4.0, 117.96927999994637, ..."
2999,3,60,20210122,100000,273.00,273.30,271.16,272.06,12095500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.55,-1,1.0,-1.0,-2.34,-2.93,-2.88,-2.50,-3.10,-3.69,-1.48,-2.35,-2.98,-1,-1.0,0.0,"[117.96927999994637, 4.0, 1201.8563199999464, 0]"


In [ ]:
test = df(columns=[1,2,3])
test[1,2] =test.apply(lambda x: x{100, 200}, axis=1)


,1,2,3,"(1, 2)"


# Отображение

In [ ]:
def show_data(data, start=0, frame_len=150):

  @interact()
  def interactive_show(div=['forward', 'backward', 'both', 'no_one'], start=(0,data.shape[0]), frame_len=(150, 300)):
    ax = basic_diagram(fig_size=(30,8))
    ax = add_candles(data, ax, start=start, frame_len=frame_len)
    if div=='forward' or div=='both':
      ax = add_ticks(data, ax, 'order', start=start, frame_len=frame_len, shift=4, color=['red', 'green']) # Опережающие тики
    if div=='backward' or div=='both':
      ax = add_ticks(data, ax, 'order_2', start=start, frame_len=frame_len, shift=7, color=['magenta', 'blue']) # Отстающие тики
    #ax = add_ticks(data, ax, 'order_3', start=start, frame_len=frame_len, shift=9, color=['grey', 'brown']) # Отстающие тики

    ax1 = basic_diagram(fig_size=(30,4))
    ax1 = add_lines(data, ax1, 'money_in', start=start, frame_len=frame_len, color='green')
    ax1 = add_lines(data, ax1, 'money_out', start=start, frame_len=frame_len, color='blue')
    #ax1 = add_lines(data, ax1, 'shares', start=start, frame_len=frame_len, color='blue')

    show_diagram()
show_data(data)

interactive(children=(Dropdown(description='div', options=('forward', 'backward', 'both', 'no_one'), value='fo…

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Трейдинг/data'
file_name = f'{path}/trends.csv'
data.to_csv(file_name, sep=';', decimal=',')